In [1]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.1 MB/s eta 0:00:00


In [2]:
!pip install -U transformers trl accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires

In [3]:
!pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [4]:
import os
import torch
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, PeftConfig
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging)


In [5]:
model_identifier="Qwen/Qwen2.5-3B-Instruct"
source_dataset="gamino/wiki_medical_terms"

In [6]:
lora_hyper_r=64
lora_hyper_alpha=16
lora_hyper_dropout=0.1

In [7]:
enable_4bit=True
compute_dtype_bnb="float16"
quant_type_bnb="nf4"
double_quant_flag=False

In [8]:
training_arguments=TrainingArguments(output_dir="./results", per_device_train_batch_size=1,gradient_accumulation_steps=4,warmup_steps=10,max_steps=100,learning_rate=5e-5,fp16=True,logging_steps=1,optim="paged_adamw_8bit")


In [9]:
enable_packing = False
sequence_length_max = None
device_assignment = {"": 0}

In [10]:
dtype_computation = getattr(torch, compute_dtype_bnb)
bnb_setup = BitsAndBytesConfig(load_in_4bit = enable_4bit,
                               bnb_4bit_quant_type = quant_type_bnb,
                               bnb_4bit_use_double_quant = double_quant_flag,
                               bnb_4bit_compute_dtype = dtype_computation)

In [11]:
llama_model = AutoModelForCausalLM.from_pretrained(model_identifier, quantization_config = bnb_setup, device_map = device_assignment)
llama_model.config.use_case = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [12]:
print(llama_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-0

In [13]:
llama_tokenizer = AutoTokenizer.from_pretrained(model_identifier, trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"
llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

1

In [14]:
data = load_dataset(source_dataset)
data = data.map(lambda samples: llama_tokenizer(samples["page_title"]), batched=True)

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

wiki_medical_terms.parquet:   0%|          | 0.00/33.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

In [15]:
def formatting_func(example):
    text = f"{example['page_title'][0]} {example['page_text'][0]}"
    return [text]
formatting_func(data["train"])

['Paracetamol poisoning Paracetamol poisoning, also known as acetaminophen poisoning, is caused by excessive use of the medication paracetamol (acetaminophen). Most people have few or non-specific symptoms in the first 24 hours following overdose. These include feeling tired, abdominal pain, or nausea. This is typically followed by a couple of days without any symptoms, after which yellowish skin, blood clotting problems, and confusion occurs as a result of liver failure. Additional complications may include kidney failure, pancreatitis, low blood sugar, and lactic acidosis. If death does not occur, people tend to recover fully over a couple of weeks. Without treatment, death from toxicity occurs 4 to 18 days later.Paracetamol poisoning can occur accidentally or as an attempt to die by suicide. Risk factors for toxicity include alcoholism, malnutrition, and the taking of certain other hepatotoxic medications. Liver damage results not from paracetamol itself, but from one of its metabol

In [16]:
peft_setup = LoraConfig(lora_alpha = lora_hyper_alpha,
                        lora_dropout = lora_hyper_dropout,
                        r = lora_hyper_r,
                        bias = "none",
                        task_type = "CAUSAL_LM",
                        target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj','lm_head'])

In [17]:
llama_sftt_trainer = SFTTrainer(model = llama_model,
                                args = training_arguments,
                                train_dataset = data["train"],
                                tokenizer = llama_tokenizer,
                                peft_config = peft_setup,
                                formatting_func=formatting_func,
                                packing = enable_packing)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:493: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [18]:
llama_sftt_trainer.train()

Step,Training Loss
1,4.989200
2,8.226700
3,4.110900
4,3.802800
5,4.496900
6,3.862900
7,6.465900
8,4.356800
9,5.829300
10,3.813400


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=100, training_loss=4.514320893287659, metrics={'train_runtime': 226.8807, 'train_samples_per_second': 1.763, 'train_steps_per_second': 0.441, 'total_flos': 37013202173952.0, 'train_loss': 4.514320893287659, 'epoch': 0.058300539279988337})

In [19]:
text = "please tell about ascariases "
device = "cuda:0"
inputs = llama_tokenizer(text, return_tensors="pt").to(device)
outputs = llama_model.generate(**inputs, max_new_tokens=200)
print(llama_tokenizer.decode(outputs[0], skip_special_tokens=True))

please tell about ascariases 1. Ascariasis is a parasitic infection caused by the roundworm Ascaris lumbricoides.
2. The worms can migrate to various organs, including the lungs, liver, and brain.
3. Common symptoms include abdominal pain, diarrhea, and coughing.
4. Diagnosis is typically made through stool examination or imaging tests.
5. Treatment involves anthelmintic medications to eliminate the worms.
6. Prevention includes proper sanitation, safe food handling, and avoiding contaminated water sources.
7. Complications can occur if the worms migrate to other organs, leading to conditions like intestinal obstruction or encephalitis. Ascariasis is a parasitic infection caused by the roundworm Ascaris lumbricoides. It is a common and widespread disease, particularly in developing countries where sanitation is inadequate. Here's a detailed overview of ascariasis:

### Causes
- **Parasite**: Ascaris lumbricoides (roundworm)
- **Transmission


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
llama_model.push_to_hub("my-fined-tuned-qwen3b",safe_serialization=False)

pytorch_model.bin:   0%|          | 0.00/3.33G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saurabh2411/my-fined-tuned-qwen3b/commit/24ea1b62e25362d13f988400e687fbc826a5b78b', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='24ea1b62e25362d13f988400e687fbc826a5b78b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saurabh2411/my-fined-tuned-qwen3b', endpoint='https://huggingface.co', repo_type='model', repo_id='Saurabh2411/my-fined-tuned-qwen3b'), pr_revision=None, pr_num=None)

In [28]:
llama_sftt_trainer.push_to_hub("Saurabh2411/my-fined-tuned-qwen3b")


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Saurabh2411/results/commit/16bee074b04115e7eda808be3b492d9bbe26ef13', commit_message='Saurabh2411/my-fined-tuned-qwen3b', commit_description='', oid='16bee074b04115e7eda808be3b492d9bbe26ef13', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saurabh2411/results', endpoint='https://huggingface.co', repo_type='model', repo_id='Saurabh2411/results'), pr_revision=None, pr_num=None)